In [ ]:
"""
IMAGE to TEXT STEGANOGRAPHY (Glyph Perturbation Cardinality, GPC)

This script extends the glyph perturbation cardinality framework to
spatial visual data by embedding image content into rendered text glyphs.
Rather than modifying image pixels directly, image intensities are first
converted into bounded integer payloads compatible with the glyph-level
perturbation channel.

Each RGB image is resized to a canonical resolution and processed
channel-wise. Pixel intensities in each channel are normalized from
[0, 255] into integer values v ∈ [0, 26]. Each integer value is then
embedded into a single text glyph by perturbing exactly v interior ink
pixels within its rasterized representation.

The raster embedding and decoding mechanisms are identical to the
text-to-text case; only the payload formation differs.

Decoding is performed by re-rasterizing the cover text, recovering
perturbation counts per glyph, inverse-normalizing the integer payloads,
and reconstructing approximate RGB image channels.

This experiment demonstrates:
- Deterministic and stable glyph-level embedding of spatial data
- High-fidelity image reconstruction under aggressive low-cardinality
  quantization
- Preservation of glyph contours and visual text appearance
- That rendered text can act as a covert carrier for image information
  without introducing visible artifacts

"""

# IMPORTING LIBRARIES
import os
import random
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from skimage.metrics import structural_similarity as ssim


# CONFIGURATION (MAIN PAPER SETTINGS)
IMAGE_PATH = "Image1.jpg"   #Input Image
OUT_DIR = "image_to_text_result"   # Output Directory
os.makedirs(OUT_DIR, exist_ok=True)

# Deterministic rendering parameters
TILE_W, TILE_H = 40, 50
FONT_SIZE = 36
CANON_SIZE = (256, 256)  # image resolution

# Payload constraints
P_MAX = 26
SEED = 42

# Grayscale convention
BG = 255
INK = 0
DELTA = 1

# Cover Text
COVER_TEXT = (
    "Once upon a misty autumn eve, a lone fox named Ember discovered an ancient silver key "
    "glowing beneath crimson leaves in the whispering woods. "
) * 2500

# ============================================================
# COVER TEXT UTILIZATION REPORT (CLARIFIED)
# ============================================================

channels = 3  # R, G, B
glyphs_per_channel = used_per_channel
total_payload_glyphs = glyphs_per_channel * channels
unused = total_cover_glyphs - glyphs_per_channel
utilization_pct = (glyphs_per_channel / total_cover_glyphs) * 100.0

print("\n===== COVER TEXT UTILIZATION =====")
print(f"Total alphabetic cover glyphs available : {total_cover_glyphs}")
print(f"Glyphs required per channel             : {glyphs_per_channel}")
print(f"Channels encoded                        : {channels}")
print(f"Total payload glyphs (R+G+B)            : {total_payload_glyphs}")
print(f"Unused cover glyphs                     : {unused}")
print(f"Cover utilization (per channel)         : {utilization_pct:.2f}%")
print("Reuse model                             : SAME glyph positions reused per channel")
print("===================================")

with open(f"{OUT_DIR}/cover_usage.txt", "w", encoding="utf-8") as f:
    f.write("IMAGE TO TEXT (GPC) - COVER TEXT UTILIZATION\n")
    f.write(f"Total alphabetic cover glyphs : {total_cover_glyphs}\n")
    f.write(f"Glyphs required per channel   : {glyphs_per_channel}\n")
    f.write(f"Channels encoded              : {channels}\n")
    f.write(f"Total payload glyphs          : {total_payload_glyphs}\n")
    f.write(f"Unused cover glyphs           : {unused}\n")
    f.write(f"Utilization per channel (%)   : {utilization_pct:.2f}\n")
    f.write("Reuse model                   : SAME glyph positions reused per channel\n")


# FONT LOADING (robust loading)
def load_font(size: int):
    for p in ["arial.ttf", "DejaVuSansMono.ttf"]:
        try:
            return ImageFont.truetype(p, size)
        except:
            pass
    return ImageFont.load_default()

FONT = load_font(FONT_SIZE)

# METRICS (IMAGE DOMAIN)
def mse(a: np.ndarray, b: np.ndarray) -> float:
    diff = a.astype(np.float64) - b.astype(np.float64)
    return float(np.mean(diff * diff))

def mae(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.mean(np.abs(a.astype(np.float64) - b.astype(np.float64))))

def rmse(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.sqrt(mse(a, b)))

def psnr(a: np.ndarray, b: np.ndarray, max_val: float = 255.0) -> float:
    m = mse(a, b)
    if m == 0:
        return float("inf")
    return float(20.0 * np.log10(max_val / np.sqrt(m)))

def snr_db(a: np.ndarray, b: np.ndarray) -> float:
    """Image SNR in dB: 10 log10( sum(signal^2) / sum(error^2) )."""
    a64 = a.astype(np.float64)
    e64 = (a.astype(np.float64) - b.astype(np.float64))
    num = np.sum(a64 * a64)
    den = np.sum(e64 * e64) + 1e-12
    return float(10.0 * np.log10(num / den))

def ssim_rgb(a: np.ndarray, b: np.ndarray):
    """
    SSIM computed per channel; returns (avg, (r,g,b)).
    Note: skimage's ssim expects 2D arrays for each channel.
    """
    sr = float(ssim(a[:, :, 0], b[:, :, 0], data_range=255))
    sg = float(ssim(a[:, :, 1], b[:, :, 1], data_range=255))
    sb = float(ssim(a[:, :, 2], b[:, :, 2], data_range=255))
    return float((sr + sg + sb) / 3.0), (sr, sg, sb)


# IMAGE to PAYLOAD
def image_to_payload(path: str):
    img = Image.open(path).convert("RGB")
    img = img.resize(CANON_SIZE, Image.BILINEAR)
    arr = np.asarray(img)

    payload = {}
    for i, ch in enumerate(["R", "G", "B"]):
        payload[ch] = np.rint(arr[:, :, i] / 255.0 * P_MAX).astype(int).flatten()
    return payload, arr


# COVER TEXT
def extract_cover_letters(text: str):
    return [c for c in text.upper() if c.isalpha()]

def cover_letters(text: str, n: int):
    letters = extract_cover_letters(text)
    if len(letters) < n:
        raise ValueError(
            f"Cover text too short: need {n} alphabetic glyphs, have {len(letters)}."
        )
    return letters[:n], len(letters)


# GLYPH RASTER
def rasterize_glyph(ch: str) -> np.ndarray:
    img = Image.new("L", (TILE_W, TILE_H), BG)
    d = ImageDraw.Draw(img)
    bbox = d.textbbox((0, 0), ch, font=FONT)
    w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
    d.text(((TILE_W - w) // 2, (TILE_H - h) // 2), ch, fill=INK, font=FONT)
    return np.asarray(img, np.uint8)

def encode_glyph(can: np.ndarray, v: int, rng: random.Random) -> np.ndarray:
    enc = can.copy()
    pts = list(zip(*np.where(can == INK)))
    for r, c in rng.sample(pts, min(v, len(pts))):
        enc[r, c] = INK + DELTA
    return enc

def decode_glyph(can: np.ndarray, enc: np.ndarray) -> int:
    return int(((can == INK) & (enc == INK + DELTA)).sum())

def arrange_tiles(tiles, per_row: int = 80) -> np.ndarray:
    blank = np.full((TILE_H, TILE_W), BG, np.uint8)
    rows = []
    for i in range(0, len(tiles), per_row):
        row = tiles[i:i + per_row]
        if len(row) < per_row:
            row += [blank] * (per_row - len(row))
        rows.append(np.hstack(row))
    return np.vstack(rows)


# CHANNEL PIPELINE
def encode_channel(vals: np.ndarray):
    letters, total_cover_letters = cover_letters(COVER_TEXT, len(vals))
    rng = random.Random(SEED)

    can_tiles, enc_tiles = [], []
    for ch, v in zip(letters, vals):
        can = rasterize_glyph(ch)
        enc = encode_glyph(can, int(v), rng)
        can_tiles.append(can)
        enc_tiles.append(enc)

    return can_tiles, enc_tiles, total_cover_letters

def decode_channel(can_tiles, enc_tiles) -> np.ndarray:
    return np.array([decode_glyph(c, e) for c, e in zip(can_tiles, enc_tiles)], dtype=int)


# MAIN PIPELINE
payload, original = image_to_payload(IMAGE_PATH)
decoded_channels = {}

total_cover_glyphs = None
used_per_channel = None

for ch in ["R", "G", "B"]:
    can_tiles, enc_tiles, cover_count = encode_channel(payload[ch])
    dec_vals = decode_channel(can_tiles, enc_tiles)

    # utilization bookkeeping (same for all channels)
    if total_cover_glyphs is None:
        total_cover_glyphs = cover_count
    if used_per_channel is None:
        used_per_channel = len(dec_vals)

    # reconstruct decoded channel in 0..255 domain
    decoded = (dec_vals.reshape(CANON_SIZE) / P_MAX * 255.0).astype(np.uint8)
    decoded_channels[ch] = decoded

    # save canonical / encoded rasters (tiled)
    Image.fromarray(arrange_tiles(can_tiles)).save(f"{OUT_DIR}/canonical_{ch}.png")
    Image.fromarray(arrange_tiles(enc_tiles)).save(f"{OUT_DIR}/encoded_{ch}.png")

    # save difference (binary perturbation map)
    diff_tiles = [
        ((can == INK) & (enc == INK + DELTA)).astype(np.uint8)
        for can, enc in zip(can_tiles, enc_tiles)
    ]
    diff_img = arrange_tiles(diff_tiles) * 255
    Image.fromarray(diff_img).save(f"{OUT_DIR}/difference_{ch}.png")


decoded_rgb = np.stack([decoded_channels[c] for c in ["R", "G", "B"]], axis=-1)

Image.fromarray(original).save(f"{OUT_DIR}/original.png")
Image.fromarray(decoded_rgb).save(f"{OUT_DIR}/decoded.png")

# METRICS REPORT (IMAGE DOMAIN)
M = mse(original, decoded_rgb)
A = mae(original, decoded_rgb)
R = rmse(original, decoded_rgb)
P = psnr(original, decoded_rgb)
SNR = snr_db(original, decoded_rgb)
SS_avg, (SS_r, SS_g, SS_b) = ssim_rgb(original, decoded_rgb)

with open(f"{OUT_DIR}/metrics.txt", "w", encoding="utf-8") as f:
    f.write("IMAGE → TEXT (GPC) METRICS (0–26)\n")
    f.write(f"Image: {IMAGE_PATH}\n")
    f.write(f"Resolution: {CANON_SIZE[0]}x{CANON_SIZE[1]}\n")
    f.write(f"P_MAX: {P_MAX}, DELTA: {DELTA}, SEED: {SEED}\n\n")

    f.write(f"MSE  : {M}\n")
    f.write(f"MAE  : {A}\n")
    f.write(f"RMSE : {R}\n")
    f.write(f"PSNR : {P}\n")
    f.write(f"SNR  : {SNR} dB\n")
    f.write(f"SSIM avg: {SS_avg}\n")
    f.write(f"SSIM R  : {SS_r}\n")
    f.write(f"SSIM G  : {SS_g}\n")
    f.write(f"SSIM B  : {SS_b}\n")

# COVER TEXT UTILIZATION REPORT
unused = total_cover_glyphs - used_per_channel

print("\n===== COVER TEXT UTILIZATION =====")
print(f"Total alphabetic cover glyphs available : {total_cover_glyphs}")
print(f"Glyphs used for embedding (per channel) : {used_per_channel}")
print(f"Unused cover glyphs                     : {unused}")
print("===================================")

with open(f"{OUT_DIR}/cover_usage.txt", "w", encoding="utf-8") as f:
    f.write("COVER TEXT UTILIZATION\n")
    f.write(f"Total alphabetic cover glyphs : {total_cover_glyphs}\n")
    f.write(f"Used glyphs (per channel)     : {used_per_channel}\n")
    f.write(f"Unused glyphs                 : {unused}\n")


print("\n✓ Image → Text GPC completed successfully")
print("✓ Canonical, encoded, difference rasters saved per channel")
print("✓ Metrics saved to metrics.txt")
print("✓ Cover usage saved to cover_usage.txt")